## Evaluation methods
- The good ol' ML way - compare against ground truth
    - BUT: metrics like accuracy/f1 score only applies to classification-type outputs.
    - For free form outputs. how??
      
- Or use another LLM (preferably GPT-3.5/4) to grade it based on the {question} {GT answer} {Our answer} tuple.
    - We can try GCP vertex AI here later.
    - For now, use mistral-7b-f16

## Architecture
- Llama.cpp server running model at 8000/v1
- **note** openai 0.28.0 to avoid error with new API. this is to be updated once langchain is fixing this bug. [link](https://github.com/langchain-ai/langchain/issues/13171)

In [1]:
from langchain.chat_models import ChatOpenAI
from ragas.llms import LangchainLLM

inference_server_url = "http://localhost:8000/v1"

# create vLLM Langchain instance
chat = ChatOpenAI(
    model="Mistral7B-instruct",
    openai_api_key="no-key",
    openai_api_base=inference_server_url,
    max_tokens=5,
    temperature=0,
)

# use the Ragas LangchainLLM wrapper to create a RagasLLM instance
vllm = LangchainLLM(llm=chat)

In [2]:
from ragas.metrics import (
    context_precision,
    answer_relevancy,
    faithfulness,
    context_recall,
)
from ragas.metrics.critique import harmfulness

# change the LLM

faithfulness.llm = vllm
answer_relevancy.llm = vllm
context_precision.llm = vllm
context_recall.llm = vllm
harmfulness.llm = vllm


In [8]:
from datasets import load_dataset
from ragas import evaluate

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")

# answer_relevancy requires OpenAI.

# bug with harmfulness currently, kiv 
result = evaluate(
    fiqa_eval["baseline"].select(range(2)),  # showing only few for demonstration
    metrics=[faithfulness, context_precision, context_recall],
)

result


evaluating with [faithfulness]


100%|███████████████████████████████████████████████████████████████| 1/1 [01:49<00:00, 109.91s/it]
/home/jack/.cache/pypoetry/virtualenvs/openroad-assistant-BiE8-jDm-py3.10/lib/python3.10/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/jack/.cache/pypoetry/virtualenvs/openroad-assistant-BiE8-jDm-py3.10/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [context_precision]


100%|████████████████████████████████████████████████████████████████| 1/1 [01:02<00:00, 62.28s/it]
/home/jack/.cache/pypoetry/virtualenvs/openroad-assistant-BiE8-jDm-py3.10/lib/python3.10/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/jack/.cache/pypoetry/virtualenvs/openroad-assistant-BiE8-jDm-py3.10/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


evaluating with [context_recall]


100%|████████████████████████████| 1/1 [01:42<00:00, 102.51s/it]
/home/jack/.cache/pypoetry/virtualenvs/openroad-assistant-BiE8-jDm-py3.10/lib/python3.10/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/jack/.cache/pypoetry/virtualenvs/openroad-assistant-BiE8-jDm-py3.10/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


{'ragas_score': 0.0000, 'faithfulness': 1.0000, 'context_precision': 0.0000, 'context_recall': 0.0000}

In [11]:
fiqa_eval["baseline"].select(range(2))[1]

{'question': 'Can I send a money order from USPS as a business?',
 'ground_truths': ["Sure you can.  You can fill in whatever you want in the From section of a money order, so your business name and address would be fine. The price only includes the money order itself.  You can hand deliver it yourself if you want, but if you want to mail it, you'll have to provide an envelope and a stamp. Note that, since you won't have a bank record of this payment, you'll want to make sure you keep other records, such as the stub of the money order.  You should probably also ask the contractor to give you a receipt."],
 'answer': '\nYes, you can send a money order from USPS as a business. You can fill in whatever you want in the From section of the money order, including your business name and address. The price only includes the money order itself, so you will need to provide an envelope and a stamp if you want to mail it. It is important to keep records of the payment, such as the stub of the mone